# Running a multi-member hydrological ensemble on the Raven Server

Here we use birdy's WPS client to launch the GR4JCN and HMETS hydrological models on the server and analyze the outputs. 

In [1]:
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

# url = "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps"
url = "http://localhost:9099/wps"
wps = WPSClient(url)

In [2]:
# The model parameters for gr4jcn and hmets. Can either be a string of comma separated values, a list, an array or a named tuple. 
gr4jcn ='0.529, -3.396, 407.29, 1.072, 16.9, 0.947'
hmets = '9.5019, 0.2774, 6.3942, 0.6884, 1.2875, 5.4134, 2.3641, 0.0973, 0.0464, 0.1998, 0.0222, -1.0919, ' \
         '2.6851, 0.3740, 1.0000, 0.4739, 0.0114, 0.0243, 0.0069, 310.7211, 916.1947'

# Forcing files. Raven uses the same forcing files for all and extracts the information it requires for each model.
ts=TESTDATA['raven-gr4j-cemaneige-nc-ts']

# Model configuration parameters.
config = dict(
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2002, 1, 1),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    )

# Launch the WPS to get the multi-model results.  Note the "gr4jcn" and "hmets" keys.
resp=wps.raven_multi_model(ts=str(ts),gr4jcn=gr4jcn,hmets=hmets, **config) 

Let's see the diagnostics and the output hydrograph. 

In [3]:
# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy the locally. 
[hydrograph, storage, solution, diagnostics] = resp.get(asobj=True)
print(diagnostics)
print(hydrograph)

AttributeError: 'XarrayConverter' object has no attribute 'url'

The `hydrograph` and `storage` outputs are netCDF files storing the time series. These files are opened by default using `xarray`, which provides convenient and powerful time series analysis and plotting tools. 

In [ ]:
hydrograph[0].q_sim

In [4]:
hydrograph[0].q_sim.plot()
hydrograph[1].q_sim.plot()

NameError: name 'hydrograph' is not defined

In [6]:
print("Max: ", hydrograph[0].q_sim.max())
print("Mean: ", hydrograph[0].q_sim.mean())
print("Monthly means: ", hydrograph[0].q_sim.groupby('time.month').mean())

Max:  <xarray.DataArray 'q_sim' ()>
array(89.922826)
Mean:  <xarray.DataArray 'q_sim' ()>
array(24.683159)
Monthly means:  <xarray.DataArray 'q_sim' (month: 12)>
array([14.517574, 11.473847, 10.072479, 20.420102, 24.375415, 22.558126,
       23.561023, 25.30086 , 28.723886, 34.377812, 46.446956, 34.094862])
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12


/home/david/src/xarray/xarray/core/groupby.py:713: FutureWarning: Default reduction dimension will be changed to the grouped dimension in a future version of xarray. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  skipna=skipna, allow_lazy=True, **kwargs)
